## Lựa chọn đặc trưng theo các phương pháp xuôi

Lựa chọn đặc trưng theo các phương pháp xuôi bắt đầu bằng cách huấn luyện mô hình học máy cho từng đặc trưng trong tập dữ liệu và lựa chọn đặc trưng mở đầu khiến mô hình hoạt động tốt nhất theo tiêu chí đánh giá nhất định.

Ở bước thứ hai, nó tạo ra các mô hình học máy cho tất cả các tổ hợp đặc trưng đã chọn ở bước trước và đặc trưng thứ hai. Nó chọn cặp tạo ra thuật toán hoạt động tốt nhất.

Phương pháp này tiếp tục bằng cách thêm mỗi lần 1 đặc trưng vào các đặc trưng đã chọn ở các bước trước cho đến khi xác định trước tiêu chí dừng.

Về lý thuyết, các mô hình có nhiều đặc trưng hơn sẽ hoạt động tốt hơn. Thuật toán sẽ tiếp tục thêm các đặc trưng mới cho đến khi đáp ứng tiêu chí, chẳng hạn: cho đến khi chất lượng của mô hình không tăng vượt quá một ngưỡng nhất định hoặc cho đến khi lựa chọn được một số đặc trưng nhất định như được triển khai trong thư viện mà chúng ta sẽ thảo luận trong notebook này.

Ví dụ, phép đo chất lượng mô hình có thể là roc_auc cho phân loại và r^2 cho hồi quy và nó do người dùng xác định.

Lựa chọn đặc trưng theo các phương pháp xuôi được gọi là thủ tục tham lam vì nó đánh giá nhiều tổ hợp đối tượng có thể: đơn, đôi, ba,... Do đó, nó rất khó tính toán và thậm chí là không khả thi nếu không gian đặc trưng lớn.


mlxtend là một gói đặc biệt trong Python thực hiện kiểu lựa chọn đặc trưng này: http://rasbt.github.io/mlxtend/


Trong triển khai mlxtend của Lựa chọn đặc trưng theo các phương pháp xuôi, tiêu chí dừng là số lượng đặc trưng được đặt tùy ý. Do đó, việc tìm kiếm sẽ kết thúc khi chúng ta đạt được số lượng đặc trưng được chọn mong muốn.


Điều này hơi tùy ý, chúng ta có thể đang chọn một số đặc trưng gần tối ưu hoặc tương tự như vậy, một số lượng lớn các đặc trưng. Tuy nhiên, bằng cách xem xét phép đo chất lượng mà thuật toán trả về khi lựa chọn đặc trưng, chúng ta có thể có biết liệu nhiều đặc trưng hơn có thêm giá trị không.



**Lưu ý**
Nếu muốn dừng tìm kiếm bằng cách sử dụng tiêu chí khác, chúng ta sẽ phải tự viết code thuật toán :(

Chúng ta sẽ sử dụng thuật toán lựa chọn đặc trưng theo các phương pháp xuôi từ mlxtend trong tập dữ liệu phân loại và hồi quy. 

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

## Hồi quy

Hãy lặp lại quy trình nhưng trong trường hợp hồi quy với tập dữ liệu giá nhà từ Kaggle để dự đoán các mục tiêu liên tục.

In [13]:
# load tập dữ liệu
data = pd.read_csv('./datatset/house-price/houseprice.csv')
data.shape

(1460, 81)

In [14]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [15]:
data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [16]:
## Yêu cầu 1
# Thực tế, nên thực hiện lựa chọn đặc trưng sau khi tiền xử lý dữ liệu, 
# mã hóa tất cả các biến hạng mục thành số, 
# sau đó chúng ta có thể đánh giá tính tất định của mục tiêu 

# để đơn giản, chúng ta sẽ chỉ sử dụng các biến dạng số
# chọn các cột số

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
## VIẾT CODE Ở ĐÂY:
numerical_vars = list(data.select_dtypes(numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [17]:
data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


<details><summary> Gợi ý </summary>

[select_dtypes()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html)

</details>

In [18]:
# chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

### Loại các đặc trưng tương quan

Lựa chọn đặc trưng theo các phương pháp xuôi cần nhiều thời gian để chạy nên để tăng tốc độ, chúng ta sẽ giảm không gian đặc trưng bằng cách loại bỏ các đặc trưng tương quan đầu tiên.

In [19]:
# tìm và loại bỏ các đặc trưng tương quan

def correlation(dataset, threshold):
    col_corr = set()  # tập hợp tất cả tên của các cột tương quan tập hợp tất cả tên của các cột tương quan
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # cần quan tâm tới các giá trị hệ số tuyệt đối 
                colname = corr_matrix.columns[i]  # lấy tên cột 
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  3


In [21]:
# các đặc trưng tương quan đã loại bỏ
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 34), (438, 34))

In [22]:
X_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
64,65,60,NaN,9375,7,5,1997,1998,573.0,739,...,2,576,36,0,0,0,0,0,2,2009
682,683,120,NaN,2887,6,5,1996,1997,0.0,1003,...,2,307,0,0,0,0,0,0,11,2008
960,961,20,50.0,7207,5,7,1958,2008,0.0,696,...,0,117,0,0,0,0,0,0,2,2010
1384,1385,50,60.0,9060,6,5,1939,1950,0.0,204,...,1,0,0,0,0,0,0,0,10,2009
1100,1101,30,60.0,8400,2,5,1920,1950,0.0,290,...,1,0,0,0,0,0,0,0,1,2009


In [24]:
X_train.isnull().mean()

Id               0.000000
MSSubClass       0.000000
LotFrontage      0.184932
LotArea          0.000000
OverallQual      0.000000
OverallCond      0.000000
YearBuilt        0.000000
YearRemodAdd     0.000000
MasVnrArea       0.004892
BsmtFinSF1       0.000000
BsmtFinSF2       0.000000
BsmtUnfSF        0.000000
TotalBsmtSF      0.000000
1stFlrSF         0.000000
2ndFlrSF         0.000000
LowQualFinSF     0.000000
GrLivArea        0.000000
BsmtFullBath     0.000000
BsmtHalfBath     0.000000
FullBath         0.000000
HalfBath         0.000000
BedroomAbvGr     0.000000
KitchenAbvGr     0.000000
Fireplaces       0.000000
GarageCars       0.000000
WoodDeckSF       0.000000
OpenPorchSF      0.000000
EnclosedPorch    0.000000
3SsnPorch        0.000000
ScreenPorch      0.000000
PoolArea         0.000000
MiscVal          0.000000
MoSold           0.000000
YrSold           0.000000
dtype: float64

In [25]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [26]:
X_train.isnull().mean()

Id               0.0
MSSubClass       0.0
LotFrontage      0.0
LotArea          0.0
OverallQual      0.0
OverallCond      0.0
YearBuilt        0.0
YearRemodAdd     0.0
MasVnrArea       0.0
BsmtFinSF1       0.0
BsmtFinSF2       0.0
BsmtUnfSF        0.0
TotalBsmtSF      0.0
1stFlrSF         0.0
2ndFlrSF         0.0
LowQualFinSF     0.0
GrLivArea        0.0
BsmtFullBath     0.0
BsmtHalfBath     0.0
FullBath         0.0
HalfBath         0.0
BedroomAbvGr     0.0
KitchenAbvGr     0.0
Fireplaces       0.0
GarageCars       0.0
WoodDeckSF       0.0
OpenPorchSF      0.0
EnclosedPorch    0.0
3SsnPorch        0.0
ScreenPorch      0.0
PoolArea         0.0
MiscVal          0.0
MoSold           0.0
YrSold           0.0
dtype: float64

### Lựa chọn đặc trưng theo các phương pháp xuôi

Chúng ta sẽ sử dụng class SFS từ MLXtend để lựa chọn đặc trưng theo các phương pháp xuôi: http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

In [27]:

# lựa chọn đặc trưng theo các phương pháp xuôi

sfs = SFS(RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=10), 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=2)

## Yêu cầu 2
## VIẾT CODE Ở ĐÂY:
sfs = sfs.fit(np.array(X_train), np.array(y_train))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   16.6s finished

[2021-11-22 15:17:53] Features: 1/20 -- score: 0.6448864917335085[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    7.2s finished

[2021-11-22 15:18:00] Features: 2/20 -- score: 0.6946490592888617[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    4.7s finished

[2021-11-22 15:18:05] Features: 3/20 -- score: 0.732141233157488[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   

Từ các bản ghi trên, chúng ta thấy sau ~ 17 đặc trưng, việc thêm nhiều đặc trưng hơn không thực sự cải thiện chất lượng.

In [28]:
## Yêu cầu 3
# chỉ số của các cột đã chọn 
## VIẾT CODE Ở ĐÂY:
sfs.k_feature_idx_

(1, 3, 4, 5, 6, 7, 11, 12, 13, 14, 16, 17, 18, 19, 23, 24, 25, 28, 29, 30)

<details><summary> Gợi ý </summary>

Using ```k_feature_idx_``` attribute

</details>

In [30]:
## Yêu cầu 4
# các cột đã chọn
## VIẾT CODE Ở ĐÂY:
X_train.columns[list(sfs.k_feature_idx_)]

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'Fireplaces',
       'GarageCars', 'WoodDeckSF', '3SsnPorch', 'ScreenPorch', 'PoolArea'],
      dtype='object')

<details><summary> Gợi ý </summary>

Using ```k_feature_idx_``` attribute

</details>

### So sánh chất lượng của các tập con đặc trưng

In [31]:
## Yêu cầu 5
# hàm để huấn luyện rừng ngẫu nhiên và đánh giá chất lượng 

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    ## VIẾT CODE Ở ĐÂY:
    pred = rf.predict(X_train)
    print('Random Forests r2 score: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    ## VIẾT CODE Ở ĐÂY:
    pred = rf.predict(X_test)
    print('Random Forests r2 score: {}'.format(r2_score(y_test, pred)))

In [32]:
selected_feat = X_train.columns[list(sfs.k_feature_idx_)]

In [36]:
len(selected_feat)

20

In [33]:
## Yêu cầu 6
# đánh giá chất lượng của thuật toán đã xây 
# sử dụng các đặc trưng đã chọn

## VIẾT CODE Ở ĐÂY:
run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests r2 score: 0.8629621682457452
Test set
Random Forests r2 score: 0.8246015925536159


In [34]:
## Yêu cầu 7
# để so sánh, chúng ta huấn luyện rừng ngẫu nhiên sử dụng 
# tất cả các đặc trưng (trừ các đặc trưng tương quan đã bị loại bỏ) 

## VIẾT CODE Ở ĐÂY:
run_randomForests(X_train,
                  X_test,
                  y_train,y_test)

Train set
Random Forests r2 score: 0.8699152317492538
Test set
Random Forests r2 score: 0.8190809813112794


Chúng ta thấy thuật toán có 20 đặc trưng thực hiện cũng tốt như thuật toán có 24 đặc trưng.

## Lựa chọn đặc trưng theo các phương pháp ngược

Lựa chọn đặc trưng theo các phương pháp ngược bắt đầu bằng cách khớp mô hình học máy sử dụng tất cả các đặc trưng trong tập dữ liệu và xác đinh chất lượng mô hình.

Sau đó, nó huấn luyện mô hình trên tất cả các tổ hợp có thể có của tất cả các đặc trưng - 1, loại bỏ đặc trưng trả về mô hình có chất lượng thấp cao nhất khi bỏ đặc trưng đó đi.

Ở bước thứ ba, huấn luyện các mô hình trong tất cả các tổ hợp có thể của các đặc trưng còn lại từ bước hai bớt đi 1 đặc trưng và loại bỏ đặc trưng khiến mô hình hoạt động tốt nhất.

Thuật toán dừng theo một tiêu chí do người dùng xác định. Tiêu chí này có thể là chất lượng mô hình không giảm vượt quá một ngưỡng nhất định hoặc đạt tới số lượng đặc trưng đã chọn nhất định như trong triển khai mlxtend.
 

Ví dụ, phép đo chất lượng mô hình có thể là roc_auc cho phân loại và r^2 cho hồi quy và nó do người dùng xác định.

Lựa chọn đặc trưng theo các phương pháp ngược được gọi là thủ tục tham lam vì nó đánh giá tất cả các tổ hợp đặc trưng n, rồi n-1, n-2,... Do đó, nó rất khó tính toán và thậm chí là không khả thi nếu không gian đặc trưng lớn.

mlxtend là một gói đặc biệt trong Python thực hiện kiểu lựa chọn đặc trưng này: http://rasbt.github.io/mlxtend/

Trong triển khai mlxtend của Lựa chọn đặc trưng theo các phương pháp ngược, tiêu chí dừng là số lượng đặc trưng được đặt tùy ý. Do đó, việc tìm kiếm sẽ kết thúc khi chúng ta đạt được số lượng đặc trưng được chọn mong muốn.

Điều này hơi tùy ý, chúng ta có thể đang chọn một số đặc trưng gần tối ưu hoặc tương tự như vậy, một số lượng lớn các đặc trưng. Tuy nhiên, bằng cách xem xét phép đo chất lượng mà thuật toán trả về khi lựa chọn đặc trưng, chúng ta có thể có biết liệu nhiều đặc trưng hơn có thêm giá trị không.


**Lưu ý:**
Nếu muốn dừng tìm kiếm bằng cách sử dụng tiêu chí khác, chúng ta sẽ phải tự viết code thuật toán :(

Chúng ta sẽ sử dụng thuật toán lựa chọn đặc trưng theo các phương pháp ngược từ mlxtend trong tập dữ liệu phân loại và hồi quy.

## Hồi quy

Hãy lặp lại quy trình nhưng trong trường hợp hồi quy với tập dữ liệu giá nhà từ Kaggle để dự đoán các mục tiêu liên tục: giá nhà

In [37]:
# load tập dữ liệu

data = pd.read_csv('./datatset/house-price/houseprice.csv')
data.shape

(1460, 81)

In [38]:
# Thực tế, nên thực hiện lựa chọn đặc trưng sau khi tiền xử lý dữ liệu,
# mã hóa tất cả các biến hạng mục thành số,
# sau đó chúng ta có thể đánh giá tính tất định của mục tiêu

# để đơn giản, chúng ta sẽ chỉ sử dụng các biến dạng số
# chọn các cột số

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [39]:
# chia thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

### Loại các đặc trưng tương quan

Lựa chọn đặc trưng theo các phương pháp ngược cần nhiều thời gian để chạy nên để tăng tốc độ, chúng ta sẽ giảm không gian đặc trưng bằng cách loại bỏ các đặc trưng tương quan đầu tiên.

In [40]:
# tìm và loại bỏ các đặc trưng tương quan

def correlation(dataset, threshold):
    col_corr = set()  # tập hợp tất cả tên của các cột tương quan
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # cần quan tâm tới các giá trị hệ số tuyệt đối
                colname = corr_matrix.columns[i]  # lấy tên cột
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  3


In [41]:
# các đặc trưng tương quan đã loại bỏ
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 34), (438, 34))

In [42]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

### Lựa chọn đặc trưng theo các phương pháp ngược


Chúng ta sẽ sử dụng class SFS từ MLXtend để lựa chọn đặc trưng theo các phương pháp ngược: http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

In [43]:
# thuật toán lựa chọn đặc trưng theo các phương pháp ngược

sfs = SFS(RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=10), 
           k_features=20, 
           forward=False, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=2)

## Yêu cầu 8
## VIẾT CODE Ở ĐÂY:
sfs = sfs.fit(np.array(X_train), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   11.0s finished

[2021-11-22 15:25:14] Features: 33/20 -- score: 0.825434533342885[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    7.1s finished

[2021-11-22 15:25:21] Features: 32/20 -- score: 0.8269182238540728[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    4.7s finished

[2021-11-22 15:25:26] Features: 31/20 -- score: 0.8321203993856869[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done

In [44]:
## Yêu cầu 9
## VIẾT CODE Ở ĐÂY:
sfs.k_feature_idx_

(0, 3, 4, 5, 6, 7, 9, 12, 14, 16, 18, 20, 22, 23, 24, 26, 27, 28, 31, 32)

<details><summary> Gợi ý </summary>

Using ```k_feature_idx_``` attribute

</details>

In [45]:
## Yêu cầu 10
## VIẾT CODE Ở ĐÂY:
X_train.columns[list(sfs.k_feature_idx_)]

Index(['Id', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', '2ndFlrSF', 'GrLivArea',
       'BsmtHalfBath', 'HalfBath', 'KitchenAbvGr', 'Fireplaces', 'GarageCars',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'MiscVal', 'MoSold'],
      dtype='object')

<details><summary> Gợi ý </summary>

Using ```k_feature_idx_``` attribute

</details>

### So sánh chất lượng của các tập con đặc trưng

In [46]:
# hàm để huấn luyện rừng ngẫu nhiên và đánh giá chất lượng

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests r2: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests r2: {}'.format(r2_score(y_test, pred)))

In [47]:
## Yêu cầu 11
## VIẾT CODE Ở ĐÂY:
selected_feat = X_train.columns[list(sfs.k_feature_idx_)]

<details><summary> Gợi ý </summary>

Using ```k_feature_idx_``` attribute

</details>

In [48]:
# đánh giá chất lượng của thuật toán đã xây
# sử dụng các đặc trưng đã chọn

run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests r2: 0.8702345974928545
Test set
Random Forests r2: 0.8240294353877188


In [49]:
# để so sánh, chúng ta huấn luyện rừng ngẫu nhiên sử dụng
# tất cả các đặc trưng (trừ các đặc trưng tương quan đã bị loại bỏ)

run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests r2: 0.8699152317492538
Test set
Random Forests r2: 0.8190809813112794


## Tìm kiếm đầy đủ

Tìm kiếm đầy đủ tìm tập hợp con các đặc trưng tốt nhất trong số tất cả các tập hợp con đặc trưng có thể theo một phép đo đặc trưng xác định cho một thuật toán học máy nhất định.

 Ví dụ: nếu chúng ta huấn luyện hồi quy logistic và tập dữ liệu gồm 4 đặc trưng, thuật toán sẽ đánh giá tất cả **15** tổ hợp đặc trưng như sau:

- tất cả các tổ hợp có thể của 1 đặc trưng
- tất cả các tổ hợp có thể của 2 đặc trưng
- tất cả các tổ hợp có thể của 3 đặc trưng
- tất cả 4 đặc trưng

và chọn tổ hợp dẫn đến chất lượng tốt nhất (ví dụ: độ chính xác của phân loại) của hồi quy logistic.

Tìm kiếm đầy đủ là một thuật toán tham lam vì nó đánh giá tất cả các kết hợp đặc trưng có thể có. Nó rất khó tính toán và thậm chí là không khả thi nếu không gian đặc trưng lớn.

mlxtend là một gói đặc biệt trong Python thực hiện kiểu lựa chọn đặc trưng này: http://rasbt.github.io/mlxtend/

Trong triển khai mlxtend của Tìm kiếm đầy đủ, tiêu chí dừng là số lượng đặc trưng được đặt tùy ý. Do đó, việc tìm kiếm sẽ kết thúc khi chúng ta đạt được số lượng đặc trưng được chọn mong muốn.

Điều này hơi tùy ý, chúng ta có thể đang chọn một số đặc trưng gần tối ưu hoặc tương tự như vậy, một số lượng lớn các đặc trưng. Tuy nhiên, bằng cách xem xét phép đo chất lượng mà thuật toán trả về khi lựa chọn đặc trưng, chúng ta có thể có biết liệu nhiều đặc trưng hơn có thêm giá trị không.


**Lưu ý:**
Nếu muốn dừng tìm kiếm bằng cách sử dụng tiêu chí khác, chúng ta sẽ phải tự viết code thuật toán :(

Chúng ta sẽ sử dụng thuật toán lựa chọn đặc trưng theo các phương pháp xuôi từ mlxtend trong tập dữ liệu phân loại và hồi quy.

In [50]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

## Hồi quy

Hãy lặp lại quy trình nhưng trong trường hợp hồi quy với tập dữ liệu giá nhà từ Kaggle để dự đoán các mục tiêu liên tục: giá nhà

In [51]:
# load tập dữ liệu
data = pd.read_csv('./datatset/house-price/houseprice.csv')
data.shape

(1460, 81)

In [52]:
# Thực tế, nên thực hiện lựa chọn đặc trưng sau khi tiền xử lý dữ liệu,
# mã hóa tất cả các biến hạng mục thành số,
# sau đó chúng ta có thể đánh giá tính tất định của mục tiêu

# để đơn giản, chúng ta sẽ chỉ sử dụng các biến dạng số
# chọn các cột số

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [53]:
# chia thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

### Loại các đặc trưng tương quan

In [54]:
# tìm và loại bỏ các đặc trưng tương quan

def correlation(dataset, threshold):
    col_corr = set()  # tập hợp tất cả tên của các cột tương quan
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # cần quan tâm tới các giá trị hệ số tuyệt đối
                colname = corr_matrix.columns[i]  # lấy tên cột
                col_corr.add(colname)
    return col_corr

# lưu ý: chúng ta giảm ngưỡng để loại bỏ các đặc trưng
corr_features = correlation(X_train, 0.6)
print('correlated features: ', len(set(corr_features)))

correlated features:  9


In [55]:
# các đặc trưng tương quan đã bị loại
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 28), (438, 28))

In [56]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

###  Tìm kiếm đầy đủ

In [58]:
# tìm kiếm đầy đủ

# để thời gian tìm kiếm ngắn hơn cho minh họa
# chúng ta sẽ yêu cầu thuật toán thử tất cả
# các tổ hợp có thể của 10 và 11 đặc trưng

# nếu truy cập vào máy tính vào hệ thống máy tính đa lõi hoặc phân tán
# chúng ta có thể thử tìm kiếm tham lam nhiều hơn

efs = EFS(RandomForestRegressor(n_estimators=5,
                                n_jobs=4,
                                random_state=0,
                                max_depth=2),
          min_features=1,
          max_features=2,
          scoring='r2',
          print_progress=True,
          cv=2)
## Yêu cầu 12
## VIẾT CODE Ở ĐÂY:
efs = efs.fit(np.array(X_train), y_train)

Features: 406/406

Tìm thấy 406 tổ hợp đặc trưng!

In [59]:
## Yêu cầu 13
## VIẾT CODE Ở ĐÂY:
efs.best_idx_

(4, 9)

<details><summary> Gợi ý </summary>

Using ```best_idx_``` attribute

</details>

In [60]:
## Yêu cầu 14
## VIẾT CODE Ở ĐÂY:
X_train.columns[list(efs.best_idx_)]

Index(['OverallQual', 'BsmtFinSF1'], dtype='object')

<details><summary> Gợi ý </summary>

Sử dụng thuộc tính ```best_idx_```

</details>

### So sánh chất lượng của các tập con đặc trưng

In [61]:
# hàm để huấn luyện rừng ngẫu nhiên và đánh giá chất lượng

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests r2: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests r2 : {}'.format(r2_score(y_test, pred)))

In [62]:
## Yêu cầu 15
## VIẾT CODE Ở ĐÂY:
selected_feat = X_train.columns[list(efs.best_idx_)]

<details><summary> Gợi ý </summary>

Using ```best_idx_``` attribute

</details>

In [63]:
# đánh giá chất lượng của thuật toán đã xây
# sử dụng các đặc trưng đã chọn

run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests r2: 0.7598962603686706
Test set
Random Forests r2 : 0.6856450799849569


In [64]:
# để so sánh, chúng ta huấn luyện rừng ngẫu nhiên sử dụng
# tất cả các đặc trưng (trừ các đặc trưng tương quan đã bị loại bỏ)

run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests r2: 0.8454047044087043
Test set
Random Forests r2 : 0.7806708079073277


Trong trường hợp này, chất lượng giảm rất nhiều nên có các đặc trưng bổ sung là các yếu tố dự báo tốt của mục tiêu.

Tìm kiếm đầy đủ rất khó tính toán, chúng ta không thường sử dụng quy trình này cho những lý do tương tự, nhưng nếu truy cập tới các siêu máy tính thì có thể thử.
